In [1]:
import numpy as np
import os
import io
import cv2

import torch
import sys
sys.path.insert(0, '/home/toolkit/eai_urlb/InternVideo/InternVideo2/multi_modality/demo/')
sys.path.insert(0, '/home/toolkit/eai_urlb/InternVideo/InternVideo2/multi_modality')

from small_config import (Config, eval_dict_leaf)
from small_utils import (retrieve_text, _frame_from_video, setup_internvideo2)

/home/toolkit/.conda/envs/urlb_test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DropoutAddRMSNorm of flash_attn is not installed!!!


[2024-04-16 22:03:29,983] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [62]:
# video = cv2.VideoCapture('example1.mp4')
video = cv2.VideoCapture('../../../../video_samples/person_walking_video.mp4')
frames = [x for x in _frame_from_video(video)]

In [63]:
text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery.",
                   "A person playing with a kid in the street",
                   "A group of friends playing bowling.",
                   "A japanese girl eating noodles",
                   "A painting by Monet",
                   "A person lying in bed",
                   "A person lying down on the grass",
                   "A person with a hat",
                   "Playing with hat",
                   "Somebody walking",
                   "Fidget spinner"]

In [4]:
if 'intern_model' in locals():
    del intern_model
    del tokenizer
config = Config.from_file('/home/toolkit/eai_urlb/InternVideo/InternVideo2/multi_modality/demo/internvideo2_stage2_config.py')
config = eval_dict_leaf(config)
config.model.vision_encoder.num_frames = 8
config.num_frames = 8
config.num_frames_test = 8
config.model.text_encoder.pretrained = '/home/toolkit/.cache/huggingface/hub/models--bert-large-uncased/snapshots/6da4b6a26a1877e173fca3225479512db81a5e5b/'
config.model.text_encoder.config = '/home/toolkit/eai_urlb/InternVideo/InternVideo2/multi_modality/' + config.model.text_encoder.config
model_pth = '/home/toolkit/eai_urlb/InternVideo/InternVideo2/download_models/InternVideo2-stage2_1b-224p-f4.pt'
config.pretrained_path = model_pth
config['model']['vision_encoder']['pretrained'] = model_pth
intern_model, tokenizer = setup_internvideo2(config)    

load_state_dict: _IncompatibleKeys(missing_keys=['text_encoder.embeddings.position_ids', 'text_encoder.embeddings.word_embeddings.weight', 'text_encoder.embeddings.position_embeddings.weight', 'text_encoder.embeddings.token_type_embeddings.weight', 'text_encoder.embeddings.LayerNorm.weight', 'text_encoder.embeddings.LayerNorm.bias', 'text_encoder.encoder.layer.0.attention.self.query.weight', 'text_encoder.encoder.layer.0.attention.self.query.bias', 'text_encoder.encoder.layer.0.attention.self.key.weight', 'text_encoder.encoder.layer.0.attention.self.key.bias', 'text_encoder.encoder.layer.0.attention.self.value.weight', 'text_encoder.encoder.layer.0.attention.self.value.bias', 'text_encoder.encoder.layer.0.attention.output.dense.weight', 'text_encoder.encoder.layer.0.attention.output.dense.bias', 'text_encoder.encoder.layer.0.attention.output.LayerNorm.weight', 'text_encoder.encoder.layer.0.attention.output.LayerNorm.bias', 'text_encoder.encoder.layer.0.intermediate.dense.weight', 'text

In [64]:
intern_model.eval()
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=14, config=config)

# Video tensor([0.0023], device='cuda:0')
# Text tensor([-0.0008, -0.0001, -0.0013, -0.0014,  0.0005, -0.0004, -0.0004, -0.0006,
#          0.0001, -0.0003,  0.0003,  0.0012, -0.0004,  0.0007, -0.0014, -0.0017,
#         -0.0007, -0.0018, -0.0006], device='cuda:0')
# text: A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery. ~ prob: 0.4592
# text: A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner. ~ prob: 0.1335
# text: A japanese girl eating noodles ~ prob: 0.1089

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

Video tensor([0.0030], device='cuda:0')
Text tensor([-0.0008, -0.0001, -0.0013, -0.0014,  0.0005, -0.0004, -0.0004, -0.0006,
         0.0001, -0.0003,  0.0003,  0.0012, -0.0004,  0.0007, -0.0014, -0.0017,
        -0.0007, -0.0018, -0.0006, -0.0024], device='cuda:0')
text: Somebody walking ~ prob: 0.6945
text: Playing with hat ~ prob: 0.1198
text: A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride. ~ prob: 0.0297
text: A person with a hat ~ prob: 0.0245
text: A person dressed in a blue jacket shovels the snow-covered pavement outside their house. ~ prob: 0.0226
text: A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner. ~ prob: 0.0222
text: A group of friends playing bowling. ~ prob: 0.0212
text: A person lying in bed ~ prob: 0.0208
text: A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery. ~ prob: 0.0186
text: A person in a blue jacket walks their pet o

: 

In [46]:
"""
Parameter containing:
tensor([[[[[-4.7913e-03, -2.1515e-03, -2.0447e-03,  ...,  3.2997e-04,
            -3.0212e-03, -7.9727e-04],
           [ 9.7656e-04,  2.4567e-03,  9.8419e-04,  ..., -1.8845e-03,
             2.3193e-03,  3.6621e-03],
           [-3.5095e-04,  2.5940e-03, -2.7618e-03,  ..., -3.7956e-04,
            -3.1948e-05,  1.7166e-03],
           ...,
           [-3.8605e-03, -1.1215e-03, -9.0790e-04,  ...,  6.5994e-04,
             1.0071e-03,  1.2894e-03],
           [-2.2278e-03,  1.2589e-03, -1.0204e-04,  ...,  3.7079e-03,
             1.5354e-04, -8.3160e-04],
           [ 6.3324e-04,  1.4114e-03,  9.5367e-04,  ..., -3.4485e-03,
            -1.8234e-03, -4.0283e-03]]],
"""
intern_model.vision_encoder.patch_embed.proj.weight


Parameter containing:
tensor([[[[[-4.7913e-03, -2.1515e-03, -2.0447e-03,  ...,  3.2997e-04,
            -3.0212e-03, -7.9727e-04],
           [ 9.7656e-04,  2.4567e-03,  9.8419e-04,  ..., -1.8845e-03,
             2.3193e-03,  3.6621e-03],
           [-3.5095e-04,  2.5940e-03, -2.7618e-03,  ..., -3.7956e-04,
            -3.1948e-05,  1.7166e-03],
           ...,
           [-3.8605e-03, -1.1215e-03, -9.0790e-04,  ...,  6.5994e-04,
             1.0071e-03,  1.2894e-03],
           [-2.2278e-03,  1.2589e-03, -1.0204e-04,  ...,  3.7079e-03,
             1.5354e-04, -8.3160e-04],
           [ 6.3324e-04,  1.4114e-03,  9.5367e-04,  ..., -3.4485e-03,
            -1.8234e-03, -4.0283e-03]]],


         [[[ 7.0190e-04, -1.3657e-03, -6.5994e-04,  ...,  1.4725e-03,
            -8.5831e-04,  1.6212e-04],
           [ 1.7262e-04,  8.0872e-04,  4.1485e-05,  ..., -6.4850e-04,
             5.5695e-04,  1.7242e-03],
           [ 1.3504e-03,  3.2959e-03, -1.3275e-03,  ...,  2.2736e-03,
             4.2

In [7]:
"""
Parameter containing:
tensor([[ 0.0366,  0.0135,  0.0492,  ..., -0.0274,  0.0493,  0.0242],
        [-0.0113,  0.0504,  0.0469,  ..., -0.0269, -0.0224, -0.0305],
        [ 0.0192, -0.0152,  0.0119,  ...,  0.0115, -0.0182, -0.0063],
        ...,
        [-0.0370, -0.0460,  0.0203,  ...,  0.0157, -0.0529,  0.0139],
        [-0.0523, -0.0192, -0.0612,  ..., -0.0515,  0.0169,  0.0098],
        [ 0.0277, -0.0029, -0.0349,  ...,  0.0014, -0.0453,  0.0052]],
       device='cuda:0')
"""
intern_model.text_encoder.encoder.layer[0].output.dense.weight

Parameter containing:
tensor([[ 0.0366,  0.0135,  0.0492,  ..., -0.0274,  0.0493,  0.0242],
        [-0.0113,  0.0504,  0.0469,  ..., -0.0269, -0.0224, -0.0305],
        [ 0.0192, -0.0152,  0.0119,  ...,  0.0115, -0.0182, -0.0063],
        ...,
        [-0.0370, -0.0460,  0.0203,  ...,  0.0157, -0.0529,  0.0139],
        [-0.0523, -0.0192, -0.0612,  ..., -0.0515,  0.0169,  0.0098],
        [ 0.0277, -0.0029, -0.0349,  ...,  0.0014, -0.0453,  0.0052]],
       device='cuda:0')

In [8]:
# Video tensor([0.0005], device='cuda:0')
# Text tensor([-1.1985e-03,  5.7084e-04, -7.3242e-05, -2.1923e-04,  1.3280e-03,
#          6.7617e-05, -5.6482e-04,  1.3007e-03,  
#      9.1326e-04,  5.7684e-04],
#        device='cuda:0')
# text: A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees. ~ prob: 0.5572
# text: A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys. ~ prob: 0.1044
# text: A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon. ~ prob: 0.0958
# text: A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride. ~ prob: 0.0936
# text: A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run. ~ prob: 0.0404

In [9]:
# /home/toolkit/.conda/envs/urlb_test/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
#   warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
# Video tensor([-0.0014], device='cuda:0')
# Text tensor([-1.8553e-03, -1.8098e-03,  5.9901e-04, -1.9457e-03,  4.7702e-05,
#         -2.8283e-03, -2.2676e-03,  7.7966e-04, -2.1556e-04, -3.8074e-04],
#        device='cuda:0')
# text: A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride. ~ prob: 0.3186
# text: A playful dog slides down a snowy hill, wagging its tail with delight. ~ prob: 0.1871
# text: A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner. ~ prob: 0.1405
# text: A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys. ~ prob: 0.1344
# text: A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run. ~ prob: 0.0955